In [46]:
import os
import sys
import pandas as pd
from DB_connection.connection import PostgresConnection
from src.utils import missing_values_table, convert_bytes_to_megabytes

In [47]:
# Add the path to the directory containing your project to the Python path
project_path = '/home/ted/Desktop/DEV/telecom-user-behavior-analysis'
sys.path.append(project_path)

In [48]:
# Change the current working directory to the project directory
os.chdir(project_path)

In [49]:
# Connect to the PostgreSQL database
db = PostgresConnection(dbname='postgres_db', user='postgres_username', password='postgres_password')
db.connect()

# Execute query to retrieve data
query = "SELECT * FROM xdr_data"
result = db.execute_query(query)

# Convert result to Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df.head())  # Display the first few rows of the DataFrame

# Close the database connection
db.close_connection()

Connected to PostgreSQL database!
Connection closed.


In [50]:
# Perform initial data analysis
missing_values_df = missing_values_table(df)
print("Missing Values in df:")
print(missing_values_df)

Your selected dataframe has 55 columns.
There are 41 columns that have missing values.
Missing Values in df:
                                          Missing Values  % of Total Values  \
Nb of sec with 37500B < Vol UL                    130254               86.8   
Nb of sec with 6250B < Vol UL < 37500B            111843               74.6   
Nb of sec with 125000B < Vol DL                    97538               65.0   
TCP UL Retrans. Vol (Bytes)                        96649               64.4   
Nb of sec with 31250B < Vol DL < 125000B           93586               62.4   
Nb of sec with 1250B < Vol UL < 6250B              92894               61.9   
Nb of sec with 6250B < Vol DL < 31250B             88317               58.9   
TCP DL Retrans. Vol (Bytes)                        88146               58.8   
HTTP UL (Bytes)                                    81810               54.5   
HTTP DL (Bytes)                                    81474               54.3   
Avg RTT DL (ms)       

In [51]:
df['HTTP DL (MB)'] = convert_bytes_to_megabytes(df, 'HTTP DL (Bytes)')

print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube UL (Bytes)  Netflix DL (Bytes)  \
0  9.16456699548519E+015  ...           2501332.0           8198936.0   
1                L77566A  ...          19111729.0          18338413.0   

In [53]:
print(df.columns)


Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [56]:
# Group the data by user identifier
user_grouped = df.groupby('IMSI')

# Calculate the number of xDR sessions for each user
user_sessions = user_grouped.size().reset_index(name='NumSessions')

# Compute the total session duration for each user
user_session_duration = user_grouped['Dur. (ms)'].sum().reset_index(name='TotalSessionDuration')

# Sum the download (DL) and upload (UL) data for each user
user_data_usage = user_grouped[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()

# Merge the dataframes to get the total data volume for each user
user_data_volume = pd.merge(user_session_duration, user_data_usage, on='IMSI')

# Calculate the total data volume (in bytes) during each session for each application
applications = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']
for app in applications:
    app_dl_col = f'{app} DL (Bytes)'
    app_ul_col = f'{app} UL (Bytes)'
    user_data_volume[app] = user_grouped[app_dl_col].fillna(0) + user_grouped[app_ul_col].fillna(0)

# Print the aggregated data
print(user_data_volume.head())




/tmp/ipykernel_17086/3971610901.py:21: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  user_data_volume[app] = user_grouped[app_dl_col].fillna(0) + user_grouped[app_ul_col].fillna(0)
/tmp/ipykernel_17086/3971610901.py:21: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  user_data_volume[app] = user_grouped[app_dl_col].fillna(0) + user_grouped[app_ul_col].fillna(0)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7757dea9cf10>>
Traceback (most recent call last):
  File "/home/ted/Desktop/DEV/telecom-user-behavior-analysis/venv/lib/python3.11/site-pac